In [1]:
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import datetime as dt
import praw
import requests
import json
from supabase import create_client, Client
import time
import requests
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

SUPABASE_URL = os.environ["SUPABASE_URL"]
SUPABASE_KEY = os.environ["SUPABASE_KEY"]
SERVICE_ROLE_KEY = os.environ["SUPABASE_SERVICE_ROLE_KEY"]

supabase: Client = create_client(SUPABASE_URL, SERVICE_ROLE_KEY)

## Get competitor data

In [2]:
# ✅ Function to Fetch Data from Supabase
def fetch_data(table_name, batch_size=500, filters=None, related_tables=None):
    try:
        all_data = []
        start = 0

        # Build select string
        if related_tables:
            select_string = "*, " + ", ".join(f"{tbl}(*)" for tbl in related_tables)
        else:
            select_string = "*"

        while True:
            query = supabase.table(table_name).select(select_string)
            
            if filters:
                for column, value in filters.items():
                    if isinstance(value, list):
                        query = query.in_(column, value)
                    elif value is None:
                        query = query.is_(column, None)
                    else:
                        query = query.eq(column, value)
            
            response = query.range(start, start + batch_size - 1).execute()
            
            if response.data:
                all_data.extend(response.data)
                start += batch_size
                if len(response.data) < batch_size:
                    break
            else:
                break

        if all_data:
            print(f"✅ Successfully fetched `{table_name}` table with filter '{filters}' and {len(all_data)} rows.")
            return pd.DataFrame(all_data)
        else:
            print(f"⚠️ `{table_name}` is empty.")
            return pd.DataFrame()

    except Exception as e:
        print(f"❌ Error fetching data from '{table_name}': {e}")
        return pd.DataFrame()

# ✅ Fetch data from tables
companies = fetch_data(
    "companies",
     filters={"status": ["trial", "active"]},
)
competitors = fetch_data(
    "competitors",
)
ads = fetch_data("ad_library")

✅ Successfully fetched `companies` table with filter '{'status': ['trial', 'active']}' and 6 rows.
✅ Successfully fetched `competitors` table with filter 'None' and 59 rows.
✅ Successfully fetched `ad_library` table with filter 'None' and 2849 rows.


In [3]:
# Filter competitors to only trial accounts
filtered_competitors = competitors[competitors["company_id"].isin(companies["id"])]

# Optional: reset index if you want a clean one
filtered_competitors = filtered_competitors.reset_index(drop=True)
competitors = filtered_competitors
print(f"✅ Filtered to {len(companies)} trial and active accounts with {len(competitors)} competitors")

✅ Filtered to 6 trial and active accounts with 25 competitors


# Facebook ads

## Grab facebook JSON response

In [4]:
url = "https://meta-facebook-ad-library.p.rapidapi.com/getPageAds"

headers = {
    "x-rapidapi-key": "e88f5d3d95msh96c8e7a091f4a90p1bee3cjsn2b537c4486ca",
    "x-rapidapi-host": "meta-facebook-ad-library.p.rapidapi.com"
}

records = []
for _, row in competitors.iterrows():
    fb_id = row.get("facebook_id")
    if pd.isna(fb_id) or fb_id == 0:
        continue

    page_id = str(int(fb_id))
    params = {
        "page_id": page_id,
        "active_status": "all",
        "ad_type": "all",
        "media_type": "all"
    }

    resp = requests.get(url, headers=headers, params=params)
    try:
        ads_json = resp.json()
    except ValueError:
        ads_json = None

    records.append({
        "competitor_id": row["id"],
        "facebook_id": page_id,
        "ads_response": ads_json
    })

# build the final DataFrame
ads_df = pd.DataFrame(records)

## Parse out Facebook JSON response

In [5]:
def parse_ads_from_df(ads_df: pd.DataFrame) -> pd.DataFrame:
    records = []

    for _, row in ads_df.iterrows():
        competitor_id = row["competitor_id"]
        response_json = row["ads_response"]

        # Skip if it's not a dict (e.g. None or raw text)
        if not isinstance(response_json, dict):
            continue

        # ads is already a list of ad‐dicts
        for ad in response_json.get("ads", []):
            ad_archive_id = ad.get("ad_archive_id")
            raw_json = json.dumps(ad)

            # Convert start_date if it's a valid Unix timestamp
            posted_at = ad.get("start_date")
            if isinstance(posted_at, (int, float)):
                posted_at = datetime.datetime.fromtimestamp(posted_at).strftime("%Y-%m-%d")

            records.append({
                "competitor_id": competitor_id,
                "ad_id": ad_archive_id,
                "json_response": raw_json,
                "postedAt": posted_at,
                "type": "meta",
            })

    return pd.DataFrame(records)

# Example usage
facebook_ads = parse_ads_from_df(ads_df)
facebook_ads.head()

,competitor_id,ad_id,json_response,postedAt,type
0,163,1653921218583398,"{""ad_archive_id"": ""1653921218583398"", ""ad_id"":...",2025-07-14,meta
1,163,531268035961251,"{""ad_archive_id"": ""531268035961251"", ""ad_id"": ...",2024-09-16,meta
2,163,1661957874784968,"{""ad_archive_id"": ""1661957874784968"", ""ad_id"":...",2025-12-03,meta
3,163,1332965254975033,"{""ad_archive_id"": ""1332965254975033"", ""ad_id"":...",2025-10-30,meta
4,163,3660326860940646,"{""ad_archive_id"": ""3660326860940646"", ""ad_id"":...",2025-07-14,meta


## Delete existing Facebook ads in supabase table

In [6]:
# # Table name
# TABLE_NAME = "ad_library"

# # Delete rows where type == "meta"
# response = supabase.table(TABLE_NAME).delete().eq("type", "meta").execute()

## Send to supabase

In [7]:
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# Convert to list of dicts
rows = facebook_ads.to_dict(orient="records")
batch_size = 500
total_upserted = 0

for batch in chunk_list(rows, batch_size):
    # specify on_conflict="ad_id"
    response = (
        supabase
        .table("ad_library")
        .upsert(batch, on_conflict="ad_id")
        .execute()
    )
    if response.data:
        print(f"✅ Inserted {len(batch)} records...")
        time.sleep(2)
    else:
        print(f"❌ Error inserting batch: {response.error}")
        break

print(f"🎉 Completed importing Facebook ads!")

✅ Inserted 20 records...
🎉 Completed importing Facebook ads!


# Google Ads

## Get ads

In [8]:
url = "https://google-ads-library.p.rapidapi.com/advertiser_ads"

headers = {
	"x-rapidapi-key": "e88f5d3d95msh96c8e7a091f4a90p1bee3cjsn2b537c4486ca",
	"x-rapidapi-host": "google-ads-library.p.rapidapi.com"
}

records = []
for _, row in competitors.iterrows():
    google_id = row.get("google_ads_id")
    if pd.isna(google_id) or str(google_id).lower() == "none":
        continue

    # ensure it's a string with no decimal
    advertiser_id = str(google_id)
    print(f"Searching for... {advertiser_id}")

    params = {"advertiser_id":google_id,"country_code":"US","format":"ALL","limit":"20"}

    resp = requests.get(url, headers=headers, params=params)
    try:
        ads_json = resp.json()
    except ValueError:
        ads_json = None 

    records.append({
        "competitor_id": row["id"],
        "google_ads_id": google_id,
        "response_json": ads_json
    })

google_ads_df = pd.DataFrame(records)

Searching for... AR03382551659182817281
Searching for... AR12657824252641148929
Searching for... AR04099081465491357697
Searching for... 0
Searching for... 0
Searching for... AR18044909407396954113
Searching for... AR16542392064705298433
Searching for... AR12871690310899466241
Searching for... AR07389967871058640897
Searching for... AR03994400846061240321
Searching for... AR11345793554250203137
Searching for... AR04810561283612999681
Searching for... AR12478872043661557761
Searching for... AR00871924403137413121
Searching for... AR17264116582418743297
Searching for... AR15861610164554563585
Searching for... AR06850986051352133633
Searching for... AR12999362336941670401
Searching for... 0
Searching for... AR02425044569474203649
Searching for... AR14896030700992987137
Searching for... AR07137559944517124097
Searching for... AR00596762134998679553


## Parse out Google ad response

In [9]:
def parse_ads_from_df(google_ads_df):
    records = []

    for _, row in google_ads_df.iterrows():
        competitor_id = row.get("competitor_id")
        rj = row.get("response_json")

        # if rj is a string, try to load it
        if isinstance(rj, str):
            try:
                rj = json.loads(rj)
            except Exception:
                continue
        # skip if still not a dict
        if not isinstance(rj, dict):
            continue

        ads = rj.get("ads")
        if not isinstance(ads, list):
            continue

        for ad in ads:
            if not isinstance(ad, dict):
                continue
            records.append({
                "competitor_id": competitor_id,
                "ad_id": ad.get("creative_id"),
                "json_response": json.dumps(ad, ensure_ascii=False),
                "postedAt": ad.get("start"),
                "type": "google",
            })

    return pd.DataFrame(records)

# usage
google_ads = parse_ads_from_df(google_ads_df)

## Send to supabase

In [10]:
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

rows = google_ads.to_dict(orient="records")
batch_size = 500
total_inserted = 0

for batch in chunk_list(rows, batch_size):
    response = supabase.table("ad_library").upsert(batch, on_conflict="ad_id").execute()
    if response.data:
        total_inserted += len(batch)
        print(f"✅ Inserted/Updated {len(batch)} records...")
        time.sleep(2)
    else:
        print(f"❌ Error inserting batch: {response.error}")
        break

print(f"🎉 Completed importing Google ads!")

✅ Inserted/Updated 287 records...
🎉 Completed importing Google ads!


# Linkedin

## Get linkedin ads

In [11]:
URL = "https://api.adyntel.com/linkedin"

HEADERS = {
    "Content-Type": "application/json"
}

API_KEY = "hd-e31ccdb6d7331807a4-e"
EMAIL = "nate@patent355.com"

records = []

for _, row in competitors.iterrows():
    competitor_name = row.get("competitor_name")
    competitor_id = row["id"]
    linkedin_id = row["linkedin_id"]

    # Skip bad or missing LinkedIn IDs
    if linkedin_id in [0, "0"] or pd.isna(linkedin_id) or str(linkedin_id).strip().lower() in ["none", "null", ""]:
        print(f"⚠️ Skipping {competitor_name} (no LinkedIn ID)")
        continue

    print(f"\n🔍 Fetching ads for {competitor_name} (linkedin_id={linkedin_id})...")

    payload = {
        "api_key": API_KEY,
        "email": EMAIL,
        "linkedin_page_id": str(linkedin_id)
    }

    try:
        resp = requests.post(URL, json=payload, headers=HEADERS, timeout=15)
        resp.raise_for_status()

        try:
            ads_json = resp.json()
        except ValueError:
            ads_json = None

        total_ads = ads_json.get("total_ads") if isinstance(ads_json, dict) else None
        print(f"   ✅ Retrieved {total_ads} ads for {competitor_name}")

        # Print first ad found (if exists)
        ads_list = ads_json.get("ads") if isinstance(ads_json, dict) else None
        if ads_list and len(ads_list) > 0:
            print("   👀 Ads found...")
        else:
            print("   ⚠️ No ads returned or unexpected structure.")

    except Exception as e:
        ads_json = {"error": str(e)}
        print(f"   ❌ Error fetching ads for {competitor_name}: {e}")

    records.append({
        "competitor_id": competitor_id,
        "linkedin_id": linkedin_id,
        "response_json": ads_json
    })

linkedin_ads_df = pd.DataFrame(records)
print("\n🎉 Finished fetching all competitor ads!")

⚠️ Skipping hampr (no LinkedIn ID)

🔍 Fetching ads for Oracle Health (linkedin_id=1028)...
   ❌ Error fetching ads for Oracle Health: HTTPSConnectionPool(host='api.adyntel.com', port=443): Read timed out. (read timeout=15)
⚠️ Skipping HappyNest (no LinkedIn ID)

🔍 Fetching ads for Suki (linkedin_id=17877435)...
   ✅ Retrieved 33 ads for Suki
   👀 Ads found...

🔍 Fetching ads for Pieces (linkedin_id=10449851)...
   ✅ Retrieved 0 ads for Pieces
   ⚠️ No ads returned or unexpected structure.

🔍 Fetching ads for Ambience Healthcare (linkedin_id=77105967)...
   ✅ Retrieved 33 ads for Ambience Healthcare
   👀 Ads found...

🔍 Fetching ads for Workiz (linkedin_id=3794219)...
   ✅ Retrieved 0 ads for Workiz
   ⚠️ No ads returned or unexpected structure.

🔍 Fetching ads for Simply Wise (linkedin_id=5151868)...
   ✅ Retrieved 0 ads for Simply Wise
   ⚠️ No ads returned or unexpected structure.

🔍 Fetching ads for Turner (linkedin_id=5772)...
   ✅ Retrieved 78 ads for Turner
   👀 Ads found...

🔍 F

## Parse out response

In [12]:
expanded_rows = []

# Today's date in YYYY-MM-DD format
posted_date = dt.datetime.utcnow().strftime("%Y-%m-%d")

for _, row in linkedin_ads_df.iterrows():
    competitor_id = row["competitor_id"]
    linkedin_id = row["linkedin_id"]
    resp = row["response_json"]

    if not resp or "ads" not in resp:
        continue

    # Extract date from continuation token if present
    continuation = resp.get("continuation_token")
    
    ad_date = None
    if continuation and "-" in continuation:
        try:
            timestamp_ms = int(continuation.split("-")[1])
            ad_date = dt.datetime.utcfromtimestamp(timestamp_ms / 1000)
        except:
            ad_date = None

    for ad in resp["ads"]:
        expanded_rows.append({
            "competitor_id": competitor_id,
            # "linkedin_in": linkedin_id,
            "ad_id": ad.get("ad_id"),
            "postedAt": posted_date,
            "type": "linkedin",
            "json_response": ad
        })

linkedin_ads = pd.DataFrame(expanded_rows)
linkedin_ads.head()

/var/folders/gj/f802y1hn0h5gfzy596h_bppr0000gn/T/ipykernel_40576/4046628761.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  posted_date = dt.datetime.utcnow().strftime("%Y-%m-%d")
/var/folders/gj/f802y1hn0h5gfzy596h_bppr0000gn/T/ipykernel_40576/4046628761.py:21: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ad_date = dt.datetime.utcfromtimestamp(timestamp_ms / 1000)


,competitor_id,ad_id,postedAt,type,json_response
0,80,816314386,2026-01-23,linkedin,"{'creative_type': 'SPONSORED_STATUS_UPDATE', '..."
1,80,1025092246,2026-01-23,linkedin,"{'creative_type': 'SPONSORED_VIDEO', 'ad_id': ..."
2,80,699571654,2026-01-23,linkedin,"{'creative_type': 'SPONSORED_STATUS_UPDATE', '..."
3,80,1046523656,2026-01-23,linkedin,"{'creative_type': 'SPONSORED_STATUS_UPDATE', '..."
4,80,887444306,2026-01-23,linkedin,"{'creative_type': 'SPONSORED_STATUS_UPDATE', '..."


In [13]:
total_duplicates = linkedin_ads["ad_id"].duplicated().sum()
print("Total duplicates:", total_duplicates)

Total duplicates: 0


## Send to supabase

In [14]:
# Deduplicate before inserting
linkedin_ads = linkedin_ads.drop_duplicates(subset=["ad_id"])

def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# Convert to list of dicts
rows = linkedin_ads.to_dict(orient="records")
batch_size = 500
total_upserted = 0

for batch in chunk_list(rows, batch_size):
    response = supabase.table("ad_library") \
                       .upsert(batch, on_conflict="ad_id") \
                       .execute()

    if response.data:
        print(f"✅ Inserted records...")
        time.sleep(2)
    else:
        print(f"❌ Error inserting batch: {response.error}")
        break

print(f"🎉 Completed importing LinkedIn ads!")

✅ Inserted records...
🎉 Completed importing LinkedIn ads!
